In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
import tensorflow_datasets as tfds

%load_ext tensorboard

from os import path
import pathlib
import tempfile

In [3]:
# normalizing the images to [0, 1]
def normalize(image, label):
    return tf.cast(image, tf.float32) / 255., label

def random_crop(image):
    cropped_image = tf.image.random_crop(
        image, size=[256, 256, 3])

    return cropped_image

def random_jitter(image):
    # resizing to 286 x 286 x 3
    image = tf.image.resize(image, [286, 286],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    # randomly cropping to 256 x 256 x 3
    image = random_crop(image)

    # random mirroring
    image = tf.image.random_flip_left_right(image)

    return image

def preprocess_flowers_train(image, label):
    image = random_jitter(image)
    return image, label

# -------------------------------

def preprocess_flowers(image, label):
    image = tf.image.resize(image, [256, 256],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return image, label

def load_flowers_dataset():  
    (ds_train, ds_validation, ds_test), ds_info = tfds.load(name="tf_flowers", 
                                             with_info=True,
                                             split=['train[:70%]', 'train[70%:85%]', 'train[85%:]'],  #70/15/15 split
                                             as_supervised=True)

    ds_train = ds_train.map(normalize)    
    ds_train = ds_train.map(preprocess_flowers)
    ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
    
    ds_validation = ds_validation.map(normalize)
    ds_validation = ds_validation.map(preprocess_flowers)
    
    ds_test = ds_test.map(normalize)
    ds_test = ds_test.map(preprocess_flowers)
    
    return ds_train, ds_validation, ds_test

def load_beans_datasets():
    (ds_train, ds_validation, ds_test), ds_info = tfds.load(
        'beans',
        split=['train', 'validation', 'test'],
        shuffle_files=True,
        as_supervised=True,
        with_info=True,
    )
    
    ds_train = ds_train.map(normalize)
    ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
    
    ds_validation = ds_validation.map(normalize)
    
    ds_test = ds_test.map(normalize)
    
    return ds_train, ds_validation, ds_test

# Optimization after training

## Quantization after training

1. Dynamic range quantization
2. Full integer quantization
3. Float16 quantization

### 1. Dynamic range quantization

Only the weights are converted from float to 8 bit int. 

### 2. Full integer quantization

Weights and activation outputs are quantizated. Good for microcontrolers and TPUs

In [4]:
def model_quantization(model_path, ds):
    # check if model was already in optimization folder
    if (len(model_path.split('/')[0].split('_')) == 3):
        optimized_dir = pathlib.Path(model_path.split('/')[0] + '/')
    else:
        optimized_dir = pathlib.Path(model_path.split('/')[0] + '_optimized/')
    model_name = model_path.split('/')[-1].split('.')[0]
    
    model = tf.keras.models.load_model(model_path)
    
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    
    # save converted tflite model
    tf_model_path = optimized_dir/(model_name + '.tflite')
    size = tf_model_path.write_bytes(tflite_model)
    print('Converted TFLite model ('+ str(size) +' Bytes) saved to: ' + str(tf_model_path))
    
    # 1. optimize model using dynamic range quantization
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_model_quant = converter.convert()
    
    tf_quant_model_path = optimized_dir/(model_name + '_dynamic_rage_quantization.tflite')
    size = tf_quant_model_path.write_bytes(tflite_model_quant)
    print('Dynamic range quantizatized TFLite model ('+ str(size) +' Bytes) saved to: ' + str(tf_quant_model_path))
    
    # 2. Full integer quantization
    def representative_data_gen():
        for input_value, _ in ds.batch(1).take(100):
            # Model has only one input so each data point has one element.
            yield [input_value]

    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_data_gen

    tflite_model_quant = converter.convert()
    tf_quant_model_path = optimized_dir/(model_name + '_full_integer_quantization.tflite')
    size = tf_quant_model_path.write_bytes(tflite_model_quant)
    print('Full integer quantizatized TFLite model ('+ str(size) +' Bytes) saved to: ' + str(tf_quant_model_path))
    
    # 2.1 Full integer quantization with input and output in integer too
    try:
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.representative_dataset = representative_data_gen
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        converter.inference_input_type = tf.uint8
        converter.inference_output_type = tf.uint8

        tflite_model_quant = converter.convert()
        tf_quant_model_path = optimized_dir/(model_name + '_full_integer_quantization_integer_io.tflite')
        size = tf_quant_model_path.write_bytes(tflite_model_quant)
        print('Full integer quantizatized with integer io TFLite model ('+ str(size) +' Bytes) saved to: ' + str(tf_quant_model_path))
    except:
        print('ERROR: Failed Full integer quantizatized with integer io TFLite model')
        
    # 3. float16 quantization
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_types = [tf.float16]
    
    tflite_model_quant = converter.convert()
    tf_quant_model_path = optimized_dir/(model_name + '_float16_quantization.tflite')
    size = tf_quant_model_path.write_bytes(tflite_model_quant)
    print('float16 quantizatized TFLite model ('+ str(size) +' Bytes) saved to: ' + str(tf_quant_model_path))


## Pruning and fine-tuning

1. Prune model to different sparsity  ( tf uses magnitude-based pruning )
    1. ConstantSparsity - sparsity is kept constant during training.
    2. PolynomialDecay - the degree of sparsity is changed during training.
2. Fine-tune model

https://www.machinecurve.com/index.php/2020/09/29/tensorflow-pruning-schedules-constantsparsity-and-polynomialdecay/

In [5]:
BATCH_SIZE = 64
PRUNING_EPOCHS = 10
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

In [6]:
# Some layers cannot be pruned

def prune_prunable_layers(model, pruning_params):
    """returns model for pruning with avoided non prunable layers"""
    
    # Rescaling layer cannot be pruned
    def apply_pruning_to_prunable(layer):
        if isinstance(layer, tf.keras.layers.experimental.preprocessing.Rescaling) or isinstance(layer, tf.keras.layers.experimental.preprocessing.Normalization):
            return layer
        return prune_low_magnitude(layer, **pruning_params)
    model_for_pruning = tf.keras.models.clone_model(
                                model,
                                clone_function=apply_pruning_to_prunable,
                            )
    return model_for_pruning

In [7]:
def prune_model(model_path, batch_size, pruning_epochs, ds_train, ds_validation, sparsity):
     # check if model was already in optimization folder
    if (len(model_path.split('/')[0].split('_')) == 3):
        optimized_dir = pathlib.Path(model_path.split('/')[0] + '/')
    else:
        optimized_dir = pathlib.Path(model_path.split('/')[0] + '_optimized/')
    model_name = model_path.split('/')[-1].split('.')[0]
    
    model = tf.keras.models.load_model(model_path)
    
    sparsities = [0.5, 0.75, 0.9]
    
    ds_train = ds_train.batch(batch_size)
    ds_validation = ds_validation.batch(batch_size)
    
    ds_train.cache()
    ds_validation.cache()
    
    # get number of images
    num_images = 0
    for i in ds_train.as_numpy_iterator():
        num_images+=1

    end_step = np.ceil(num_images / batch_size).astype(np.int32) * pruning_epochs

    # Define pruning configuration
    pruning_params_constant = {
        'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=sparsity,
                                                                    begin_step=0,
                                                                    end_step=end_step)
    }

    pruning_params_polynomial = {
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0,
                                                                final_sparsity=sparsity,
                                                                begin_step=0,
                                                                end_step=end_step)
    }

    # Rescaling layer cannot be wrapped in prune low magnitude
    try:
        model_for_pruning_constant = prune_low_magnitude(model, **pruning_params_constant)
        model_for_pruning_polynomial = prune_low_magnitude(model, **pruning_params_polynomial)
    except:
        model_for_pruning_constant = prune_prunable_layers(model, pruning_params_constant) 
        model_for_pruning_polynomial = prune_prunable_layers(model, pruning_params_polynomial)


    # Compile models for pruning
    model_for_pruning_constant.compile(optimizer='adam',
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])
    model_for_pruning_polynomial.compile(optimizer='adam',
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])


    constant_log_dir = pathlib.Path("./tmp/" + model_name + '_ConstantSparsity' + str(int(sparsity*100)))
    constant_log_dir.mkdir(parents=True, exist_ok=True)
    polynomial_log_dir = pathlib.Path("./tmp/" + model_name + '_PolynomialDecay' + str(int(sparsity*100)))
    polynomial_log_dir.mkdir(parents=True, exist_ok=True)

    # Model callbacks
    constant_callbacks = [
        tfmot.sparsity.keras.UpdatePruningStep(),
        tfmot.sparsity.keras.PruningSummaries(log_dir=str(constant_log_dir))
    ]
    polynomial_callbacks = [
        tfmot.sparsity.keras.UpdatePruningStep(),
        tfmot.sparsity.keras.PruningSummaries(log_dir=str(polynomial_log_dir))
    ]

    # Fitting data
    model_for_pruning_constant.fit(ds_train,
                              validation_data=ds_validation,
                              epochs=pruning_epochs,
                              callbacks=constant_callbacks)

    model_for_pruning_polynomial.fit(ds_train,
                              validation_data=ds_validation,
                              epochs=pruning_epochs,
                              callbacks=polynomial_callbacks)

    # Save pruned models
    model_constant_path = optimized_dir/(model_name + '_ConstantSparsity' + str(int(sparsity*100)) + '.h5')
    model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning_constant)
    model_for_export.save(str(model_constant_path))
    print('saved ' + str(model_constant_path))

    model_polynomial_path = optimized_dir/(model_name + '_PolynomialDecay' + str(int(sparsity*100)) + '.h5')
    model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning_polynomial)
    model_for_export.save(str(model_polynomial_path))
    print('saved ' + str(model_polynomial_path))

## Weight clustering

In [8]:
cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

In [9]:
# Some layers cannot be weight clustered

def cluster_clustred_layers(model, cluster_params):    
    # Rescaling layer cannot be pruned
    def apply_clustering_to_clusterable(layer):
        if model.layers[0] == layer or  model.layers[-1] == layer:
            return layer
        try:
            x = cluster_weights(layer, **cluster_params)
            return x
        except:
            return layer

    model_for_clustering = tf.keras.models.clone_model(
                                model,
                                clone_function=apply_clustering_to_clusterable,
                            )
    return model_for_clustering

In [10]:
def weight_cluster_model(model_path, batch_size, epochs, ds_train, ds_validation, number_of_clusters):
    """ Weight clustering on given moodel 
    note: cannot use for cycle in this function to do different number of clusters because of compatibility issues"""
     # check if model was already in optimization folder
    if (len(model_path.split('/')[0].split('_')) == 3):
        optimized_dir = pathlib.Path(model_path.split('/')[0] + '/')
    else:
        optimized_dir = pathlib.Path(model_path.split('/')[0] + '_optimized/')
    model_name = model_path.split('/')[-1].split('.')[0]
    
    model = tf.keras.models.load_model(model_path)
    
    ds_train = ds_train.batch(batch_size)
    ds_validation = ds_validation.batch(batch_size)
    
    ds_train.cache()
    ds_validation.cache()
    
    # get number of images
    num_images = 0
    for i in ds_train.as_numpy_iterator():
        num_images+=1

    end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

    # Define weight clustering configuration
    cluster_params_kmeans = {
                                  'number_of_clusters': number_of_clusters,
                                  'cluster_centroids_init': CentroidInitialization.KMEANS_PLUS_PLUS
                                }

    # Rescaling layer cannot be wrapped in prune low magnitude
    try:
        model_for_clustering_kmeans = cluster_weights(model, **cluster_params_kmeans)
    except:
        model_for_clustering_kmeans = cluster_clustred_layers(model, cluster_params_kmeans)

    # Compile models for clustering
    model_for_clustering_kmeans.compile(optimizer='adam',
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])

    # Fitting data        
    model_for_clustering_kmeans.fit(ds_train,
                              validation_data=ds_validation,
                              epochs=epochs)

    # Save pruned models
    model_kmeans_path = optimized_dir/(model_name + '_KMeansPlusPlus' + str(number_of_clusters) + '.h5')
    model_for_export = tfmot.clustering.keras.strip_clustering(model_for_clustering_kmeans)
    model_for_export.save(str(model_kmeans_path))
    print('saved ' + str(model_kmeans_path))

# optimze models

In [11]:
# datasets
flowers_datasets = load_flowers_dataset()
beans_datasets = load_beans_datasets()

### Note

Due to bad tensorflow optimization of calling fit function in a loop, calling prune_model in a loop is unusable.
Solution: don't use for loops
Issue: https://github.com/tensorflow/tensorflow/issues/34025

## 1. Prune base models

In [11]:
prune_model(model_path='flowers_models/MobileNetV2_flowers_model.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.5)
prune_model(model_path='flowers_models/MobileNetV2_flowers_model.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.75)
prune_model(model_path='flowers_models/MobileNetV2_flowers_model.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.9)

/home/macenpav/.conda/envs/ml/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Epoch 1/10
 6/41 [===>..........................] - ETA: 20s - loss: 0.3058 - accuracy: 0.9041WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1502s vs `on_train_batch_end` time: 0.3473s). Check your callbacks.


41/41 [==============================] - 54s 597ms/step - loss: 0.3580 - accuracy: 0.8871 - val_loss: 5.1500 - val_accuracy: 0.3557
Epoch 2/10
41/41 [==============================] - 38s 526ms/step - loss: 0.0960 - accuracy: 0.9710 - val_loss: 5.0288 - val_accuracy: 0.3829
Epoch 3/10
41/41 [==============================] - 37s 525ms/step - loss: 0.0344 - accuracy: 0.9891 - val_loss: 4.2880 - val_accuracy: 0.4011
Epoch 4/10
41/41 [==============================] - 36s 533ms/step - loss: 0.0588 - accuracy: 0.9813 - val_loss: 2.2261 - val_accuracy: 0.5971
Epoch 5/10
41/41 [==============================] - 37s 530ms/step - loss: 0.0705 - accuracy: 0.9781 - val_loss: 1.5959 - val_accuracy: 0.6352
Epoch 6/10
41/41 [==============================] - 35s 534ms/step - loss: 0.0371 - accuracy: 0.9872 - val_loss: 1.7022 - val_accuracy: 0.6388
Epoch 7/10
41/41 [==============================] - 36s 530ms/step - loss: 0.0238 - accuracy: 0.9932 - val_loss: 1.3656 - val_accuracy: 0.6969
Epoch 8/10

41/41 [==============================] - 58s 616ms/step - loss: 0.1085 - accuracy: 0.9670 - val_loss: 2.9428 - val_accuracy: 0.5808
Epoch 2/10
41/41 [==============================] - 34s 519ms/step - loss: 0.1731 - accuracy: 0.9413 - val_loss: 4.3531 - val_accuracy: 0.5735
Epoch 3/10
41/41 [==============================] - 32s 535ms/step - loss: 0.0723 - accuracy: 0.9759 - val_loss: 3.3222 - val_accuracy: 0.6098
Epoch 4/10
41/41 [==============================] - 37s 522ms/step - loss: 0.0518 - accuracy: 0.9831 - val_loss: 6.5352 - val_accuracy: 0.5445
Epoch 5/10
41/41 [==============================] - 32s 513ms/step - loss: 0.0987 - accuracy: 0.9665 - val_loss: 10.4302 - val_accuracy: 0.4392
Epoch 6/10
41/41 [==============================] - 35s 503ms/step - loss: 0.1381 - accuracy: 0.9529 - val_loss: 10.3451 - val_accuracy: 0.5100
Epoch 7/10
41/41 [==============================] - 35s 524ms/step - loss: 0.0583 - accuracy: 0.9836 - val_loss: 8.6702 - val_accuracy: 0.5082
Epoch 8/

41/41 [==============================] - 53s 593ms/step - loss: 1.3621 - accuracy: 0.6090 - val_loss: 3.9587 - val_accuracy: 0.2668
Epoch 2/10
41/41 [==============================] - 30s 529ms/step - loss: 0.4331 - accuracy: 0.8242 - val_loss: 4.3595 - val_accuracy: 0.2668
Epoch 3/10
41/41 [==============================] - 31s 528ms/step - loss: 0.1594 - accuracy: 0.9592 - val_loss: 4.5802 - val_accuracy: 0.2668
Epoch 4/10
41/41 [==============================] - 33s 537ms/step - loss: 0.1052 - accuracy: 0.9777 - val_loss: 4.9835 - val_accuracy: 0.2668
Epoch 5/10
41/41 [==============================] - 32s 521ms/step - loss: 0.1288 - accuracy: 0.9625 - val_loss: 5.1171 - val_accuracy: 0.2668
Epoch 6/10
41/41 [==============================] - 39s 557ms/step - loss: 0.1020 - accuracy: 0.9709 - val_loss: 5.2919 - val_accuracy: 0.2668
Epoch 7/10
41/41 [==============================] - 37s 540ms/step - loss: 0.1084 - accuracy: 0.9661 - val_loss: 5.4508 - val_accuracy: 0.2668
Epoch 8/10

41/41 [==============================] - 54s 595ms/step - loss: 0.2528 - accuracy: 0.9157 - val_loss: 6.6258 - val_accuracy: 0.2668
Epoch 2/10
41/41 [==============================] - 33s 534ms/step - loss: 0.2951 - accuracy: 0.9009 - val_loss: 5.7449 - val_accuracy: 0.2668
Epoch 3/10
41/41 [==============================] - 34s 532ms/step - loss: 0.2157 - accuracy: 0.9184 - val_loss: 5.2072 - val_accuracy: 0.2668
Epoch 4/10
41/41 [==============================] - 35s 526ms/step - loss: 0.2431 - accuracy: 0.9110 - val_loss: 5.0934 - val_accuracy: 0.2668
Epoch 5/10
41/41 [==============================] - 35s 533ms/step - loss: 0.1069 - accuracy: 0.9659 - val_loss: 6.1679 - val_accuracy: 0.2668
Epoch 6/10
41/41 [==============================] - 36s 532ms/step - loss: 0.0515 - accuracy: 0.9840 - val_loss: 5.7989 - val_accuracy: 0.2668
Epoch 7/10
41/41 [==============================] - 34s 541ms/step - loss: 0.0355 - accuracy: 0.9933 - val_loss: 5.3291 - val_accuracy: 0.2668
Epoch 8/10

41/41 [==============================] - 56s 591ms/step - loss: 2.9286 - accuracy: 0.3711 - val_loss: 3.1051 - val_accuracy: 0.2668
Epoch 2/10
41/41 [==============================] - 37s 530ms/step - loss: 1.0559 - accuracy: 0.6079 - val_loss: 2.8824 - val_accuracy: 0.2668
Epoch 3/10
41/41 [==============================] - 39s 535ms/step - loss: 0.8123 - accuracy: 0.7015 - val_loss: 2.7914 - val_accuracy: 0.2668
Epoch 4/10
41/41 [==============================] - 35s 548ms/step - loss: 0.6016 - accuracy: 0.8021 - val_loss: 2.7655 - val_accuracy: 0.2668
Epoch 5/10
41/41 [==============================] - 36s 535ms/step - loss: 0.4445 - accuracy: 0.8632 - val_loss: 2.8184 - val_accuracy: 0.2668
Epoch 6/10
41/41 [==============================] - 35s 527ms/step - loss: 0.3561 - accuracy: 0.8851 - val_loss: 2.8705 - val_accuracy: 0.2668
Epoch 7/10
41/41 [==============================] - 33s 510ms/step - loss: 0.3017 - accuracy: 0.8930 - val_loss: 2.9406 - val_accuracy: 0.2668
Epoch 8/10

41/41 [==============================] - 52s 597ms/step - loss: 0.9309 - accuracy: 0.6907 - val_loss: 2.2783 - val_accuracy: 0.2668
Epoch 2/10
41/41 [==============================] - 33s 537ms/step - loss: 0.6764 - accuracy: 0.7412 - val_loss: 3.0429 - val_accuracy: 0.2668
Epoch 3/10
41/41 [==============================] - 38s 557ms/step - loss: 0.4508 - accuracy: 0.8334 - val_loss: 3.5481 - val_accuracy: 0.2668
Epoch 4/10
41/41 [==============================] - 37s 533ms/step - loss: 0.4408 - accuracy: 0.8369 - val_loss: 3.9086 - val_accuracy: 0.2668
Epoch 5/10
41/41 [==============================] - 34s 524ms/step - loss: 0.3641 - accuracy: 0.8789 - val_loss: 3.4979 - val_accuracy: 0.2668
Epoch 6/10
41/41 [==============================] - 35s 513ms/step - loss: 0.2674 - accuracy: 0.9103 - val_loss: 4.1487 - val_accuracy: 0.2668
Epoch 7/10
41/41 [==============================] - 35s 532ms/step - loss: 0.2970 - accuracy: 0.8931 - val_loss: 4.5642 - val_accuracy: 0.2668
Epoch 8/10

In [12]:
prune_model(model_path='flowers_models/EfficentNetB0_flowers_model.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1], 
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.5)
prune_model(model_path='flowers_models/EfficentNetB0_flowers_model.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1], 
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.75)
prune_model(model_path='flowers_models/EfficentNetB0_flowers_model.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1], 
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.9)

Epoch 1/10
 6/41 [===>..........................] - ETA: 44s - loss: 0.7728 - accuracy: 0.7121WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.2461s vs `on_train_batch_end` time: 0.8278s). Check your callbacks.


41/41 [==============================] - 79s 879ms/step - loss: 0.5923 - accuracy: 0.7785 - val_loss: 1.5731 - val_accuracy: 0.4773
Epoch 2/10
41/41 [==============================] - 46s 729ms/step - loss: 0.1828 - accuracy: 0.9404 - val_loss: 1.9018 - val_accuracy: 0.4973
Epoch 3/10
41/41 [==============================] - 42s 752ms/step - loss: 0.0795 - accuracy: 0.9765 - val_loss: 2.0443 - val_accuracy: 0.5481
Epoch 4/10
41/41 [==============================] - 43s 758ms/step - loss: 0.0547 - accuracy: 0.9831 - val_loss: 1.2520 - val_accuracy: 0.6806
Epoch 5/10
41/41 [==============================] - 42s 775ms/step - loss: 0.0809 - accuracy: 0.9727 - val_loss: 1.4513 - val_accuracy: 0.6534
Epoch 6/10
41/41 [==============================] - 46s 737ms/step - loss: 0.0733 - accuracy: 0.9764 - val_loss: 1.3660 - val_accuracy: 0.6770
Epoch 7/10
41/41 [==============================] - 44s 756ms/step - loss: 0.0609 - accuracy: 0.9820 - val_loss: 1.3995 - val_accuracy: 0.6951
Epoch 8/10

41/41 [==============================] - 81s 879ms/step - loss: 0.1260 - accuracy: 0.9614 - val_loss: 3.8911 - val_accuracy: 0.4374
Epoch 2/10
41/41 [==============================] - 44s 763ms/step - loss: 0.1186 - accuracy: 0.9572 - val_loss: 3.0033 - val_accuracy: 0.5336
Epoch 3/10
41/41 [==============================] - 45s 761ms/step - loss: 0.1253 - accuracy: 0.9527 - val_loss: 2.6494 - val_accuracy: 0.6062
Epoch 4/10
41/41 [==============================] - 44s 747ms/step - loss: 0.0956 - accuracy: 0.9623 - val_loss: 2.7205 - val_accuracy: 0.5898
Epoch 5/10
41/41 [==============================] - 40s 744ms/step - loss: 0.1782 - accuracy: 0.9386 - val_loss: 3.0062 - val_accuracy: 0.5626
Epoch 6/10
41/41 [==============================] - 42s 750ms/step - loss: 0.1182 - accuracy: 0.9591 - val_loss: 1.9782 - val_accuracy: 0.6134
Epoch 7/10
41/41 [==============================] - 46s 744ms/step - loss: 0.0339 - accuracy: 0.9898 - val_loss: 1.7728 - val_accuracy: 0.6497
Epoch 8/10

41/41 [==============================] - 77s 874ms/step - loss: 1.2200 - accuracy: 0.5120 - val_loss: 1.8158 - val_accuracy: 0.2668
Epoch 2/10
41/41 [==============================] - 46s 750ms/step - loss: 0.8584 - accuracy: 0.6818 - val_loss: 1.8276 - val_accuracy: 0.2668
Epoch 3/10
41/41 [==============================] - 45s 742ms/step - loss: 0.6356 - accuracy: 0.7946 - val_loss: 1.4092 - val_accuracy: 0.3702
Epoch 4/10
41/41 [==============================] - 41s 734ms/step - loss: 0.4012 - accuracy: 0.8990 - val_loss: 1.4196 - val_accuracy: 0.4828
Epoch 5/10
41/41 [==============================] - 47s 739ms/step - loss: 0.2796 - accuracy: 0.9260 - val_loss: 1.4320 - val_accuracy: 0.5100
Epoch 6/10
41/41 [==============================] - 47s 750ms/step - loss: 0.2219 - accuracy: 0.9456 - val_loss: 1.5949 - val_accuracy: 0.5535
Epoch 7/10
41/41 [==============================] - 41s 732ms/step - loss: 0.1873 - accuracy: 0.9482 - val_loss: 1.0908 - val_accuracy: 0.6443
Epoch 8/10

41/41 [==============================] - 79s 835ms/step - loss: 0.2654 - accuracy: 0.9192 - val_loss: 1.9785 - val_accuracy: 0.5789
Epoch 2/10
41/41 [==============================] - 42s 740ms/step - loss: 0.2239 - accuracy: 0.9252 - val_loss: 2.0486 - val_accuracy: 0.5880
Epoch 3/10
41/41 [==============================] - 42s 744ms/step - loss: 0.1714 - accuracy: 0.9323 - val_loss: 2.0495 - val_accuracy: 0.6171
Epoch 4/10
41/41 [==============================] - 45s 737ms/step - loss: 0.2035 - accuracy: 0.9278 - val_loss: 1.8478 - val_accuracy: 0.6207
Epoch 5/10
41/41 [==============================] - 39s 737ms/step - loss: 0.2405 - accuracy: 0.9215 - val_loss: 4.5919 - val_accuracy: 0.4555
Epoch 6/10
41/41 [==============================] - 44s 750ms/step - loss: 0.1622 - accuracy: 0.9456 - val_loss: 1.6805 - val_accuracy: 0.6570
Epoch 7/10
41/41 [==============================] - 46s 759ms/step - loss: 0.1478 - accuracy: 0.9495 - val_loss: 1.4376 - val_accuracy: 0.6915
Epoch 8/10

41/41 [==============================] - 76s 867ms/step - loss: 1.6785 - accuracy: 0.2825 - val_loss: 2.8489 - val_accuracy: 0.2668
Epoch 2/10
41/41 [==============================] - 46s 754ms/step - loss: 1.5950 - accuracy: 0.3413 - val_loss: 2.2904 - val_accuracy: 0.2668
Epoch 3/10
41/41 [==============================] - 43s 750ms/step - loss: 1.5618 - accuracy: 0.3802 - val_loss: 1.9368 - val_accuracy: 0.2668
Epoch 4/10
41/41 [==============================] - 45s 737ms/step - loss: 1.5280 - accuracy: 0.4118 - val_loss: 1.7449 - val_accuracy: 0.2668
Epoch 5/10
41/41 [==============================] - 43s 744ms/step - loss: 1.4901 - accuracy: 0.4577 - val_loss: 1.6661 - val_accuracy: 0.2668
Epoch 6/10
41/41 [==============================] - 46s 761ms/step - loss: 1.4596 - accuracy: 0.4740 - val_loss: 1.6009 - val_accuracy: 0.2668
Epoch 7/10
41/41 [==============================] - 46s 731ms/step - loss: 1.4277 - accuracy: 0.4848 - val_loss: 1.5531 - val_accuracy: 0.2904
Epoch 8/10

41/41 [==============================] - 81s 895ms/step - loss: 1.1562 - accuracy: 0.5271 - val_loss: 4.1351 - val_accuracy: 0.4864
Epoch 2/10
41/41 [==============================] - 42s 754ms/step - loss: 0.8153 - accuracy: 0.6727 - val_loss: 4.2152 - val_accuracy: 0.4773
Epoch 3/10
41/41 [==============================] - 42s 724ms/step - loss: 0.6883 - accuracy: 0.7243 - val_loss: 1.8514 - val_accuracy: 0.5463
Epoch 4/10
41/41 [==============================] - 46s 727ms/step - loss: 0.5517 - accuracy: 0.7890 - val_loss: 3.2529 - val_accuracy: 0.4356
Epoch 5/10
41/41 [==============================] - 37s 761ms/step - loss: 0.3863 - accuracy: 0.8406 - val_loss: 1.8385 - val_accuracy: 0.6098
Epoch 6/10
41/41 [==============================] - 45s 740ms/step - loss: 0.2801 - accuracy: 0.8962 - val_loss: 3.8564 - val_accuracy: 0.4483
Epoch 7/10
41/41 [==============================] - 40s 748ms/step - loss: 0.2984 - accuracy: 0.8950 - val_loss: 1.7538 - val_accuracy: 0.6279
Epoch 8/10

In [12]:
prune_model(model_path='beans_models/MobileNetV2_beans_model.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.5)
prune_model(model_path='beans_models/MobileNetV2_beans_model.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.75)
prune_model(model_path='beans_models/MobileNetV2_beans_model.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.9)

/home/macenpav/.conda/envs/ml/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Epoch 1/10
 6/17 [=========>....................] - ETA: 20s - loss: 0.6184 - accuracy: 0.8572WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.4185s vs `on_train_batch_end` time: 1.2123s). Check your callbacks.


17/17 [==============================] - 53s 2s/step - loss: 0.5875 - accuracy: 0.8552 - val_loss: 1.5991 - val_accuracy: 0.5338
Epoch 2/10
17/17 [==============================] - 34s 2s/step - loss: 0.1781 - accuracy: 0.9296 - val_loss: 4.0393 - val_accuracy: 0.5564
Epoch 3/10
17/17 [==============================] - 32s 2s/step - loss: 0.1089 - accuracy: 0.9697 - val_loss: 4.4042 - val_accuracy: 0.4887
Epoch 4/10
17/17 [==============================] - 33s 2s/step - loss: 0.0475 - accuracy: 0.9837 - val_loss: 3.0142 - val_accuracy: 0.4962
Epoch 5/10
17/17 [==============================] - 33s 2s/step - loss: 0.0338 - accuracy: 0.9895 - val_loss: 4.3552 - val_accuracy: 0.6090
Epoch 6/10
17/17 [==============================] - 32s 2s/step - loss: 0.0246 - accuracy: 0.9916 - val_loss: 2.9429 - val_accuracy: 0.6015
Epoch 7/10
17/17 [==============================] - 33s 2s/step - loss: 0.1067 - accuracy: 0.9626 - val_loss: 1.6338 - val_accuracy: 0.6992
Epoch 8/10
17/17 [=============

17/17 [==============================] - 53s 2s/step - loss: 0.1016 - accuracy: 0.9610 - val_loss: 4.1431 - val_accuracy: 0.6316
Epoch 2/10
17/17 [==============================] - 33s 2s/step - loss: 0.1341 - accuracy: 0.9611 - val_loss: 9.6285 - val_accuracy: 0.4812
Epoch 3/10
17/17 [==============================] - 33s 2s/step - loss: 0.0675 - accuracy: 0.9806 - val_loss: 5.8350 - val_accuracy: 0.5865
Epoch 4/10
17/17 [==============================] - 33s 2s/step - loss: 0.0172 - accuracy: 0.9985 - val_loss: 9.9832 - val_accuracy: 0.5263
Epoch 5/10
17/17 [==============================] - 32s 2s/step - loss: 0.0230 - accuracy: 0.9934 - val_loss: 13.9046 - val_accuracy: 0.3684
Epoch 6/10
17/17 [==============================] - 33s 2s/step - loss: 0.0099 - accuracy: 0.9981 - val_loss: 27.3363 - val_accuracy: 0.3459
Epoch 7/10
17/17 [==============================] - 32s 2s/step - loss: 0.2814 - accuracy: 0.9215 - val_loss: 12.2386 - val_accuracy: 0.4812
Epoch 8/10
17/17 [==========

17/17 [==============================] - 51s 2s/step - loss: 1.3982 - accuracy: 0.6213 - val_loss: 1.1567 - val_accuracy: 0.3308
Epoch 2/10
17/17 [==============================] - 33s 2s/step - loss: 0.3990 - accuracy: 0.8528 - val_loss: 1.1944 - val_accuracy: 0.3308
Epoch 3/10
17/17 [==============================] - 32s 2s/step - loss: 0.2310 - accuracy: 0.9306 - val_loss: 1.2994 - val_accuracy: 0.3308
Epoch 4/10
17/17 [==============================] - 32s 2s/step - loss: 0.1158 - accuracy: 0.9729 - val_loss: 1.5247 - val_accuracy: 0.3308
Epoch 5/10
17/17 [==============================] - 32s 2s/step - loss: 0.0791 - accuracy: 0.9815 - val_loss: 1.5415 - val_accuracy: 0.3308
Epoch 6/10
17/17 [==============================] - 32s 2s/step - loss: 0.0591 - accuracy: 0.9862 - val_loss: 1.6809 - val_accuracy: 0.3308
Epoch 7/10
17/17 [==============================] - 32s 2s/step - loss: 0.0443 - accuracy: 0.9912 - val_loss: 1.8017 - val_accuracy: 0.3308
Epoch 8/10
17/17 [=============

17/17 [==============================] - 53s 2s/step - loss: 0.2393 - accuracy: 0.9108 - val_loss: 2.0910 - val_accuracy: 0.3308
Epoch 2/10
17/17 [==============================] - 32s 2s/step - loss: 0.2740 - accuracy: 0.8969 - val_loss: 1.7010 - val_accuracy: 0.3308
Epoch 3/10
17/17 [==============================] - 33s 2s/step - loss: 0.2203 - accuracy: 0.9197 - val_loss: 2.3330 - val_accuracy: 0.3308
Epoch 4/10
17/17 [==============================] - 32s 2s/step - loss: 0.1100 - accuracy: 0.9617 - val_loss: 3.7505 - val_accuracy: 0.3308
Epoch 5/10
17/17 [==============================] - 32s 2s/step - loss: 0.1012 - accuracy: 0.9688 - val_loss: 3.5479 - val_accuracy: 0.3308
Epoch 6/10
17/17 [==============================] - 32s 2s/step - loss: 0.0494 - accuracy: 0.9869 - val_loss: 3.6504 - val_accuracy: 0.3308
Epoch 7/10
17/17 [==============================] - 33s 2s/step - loss: 0.0778 - accuracy: 0.9662 - val_loss: 3.7850 - val_accuracy: 0.3308
Epoch 8/10
17/17 [=============

17/17 [==============================] - 52s 2s/step - loss: 1.2064 - accuracy: 0.4685 - val_loss: 1.1073 - val_accuracy: 0.3383
Epoch 2/10
17/17 [==============================] - 33s 2s/step - loss: 0.6349 - accuracy: 0.7247 - val_loss: 1.1016 - val_accuracy: 0.3383
Epoch 3/10
17/17 [==============================] - 33s 2s/step - loss: 0.4866 - accuracy: 0.8216 - val_loss: 1.1025 - val_accuracy: 0.3383
Epoch 4/10
17/17 [==============================] - 33s 2s/step - loss: 0.3388 - accuracy: 0.8949 - val_loss: 1.1033 - val_accuracy: 0.3383
Epoch 5/10
17/17 [==============================] - 32s 2s/step - loss: 0.2166 - accuracy: 0.9599 - val_loss: 1.1052 - val_accuracy: 0.3308
Epoch 6/10
17/17 [==============================] - 33s 2s/step - loss: 0.1505 - accuracy: 0.9632 - val_loss: 1.1058 - val_accuracy: 0.3308
Epoch 7/10
17/17 [==============================] - 32s 2s/step - loss: 0.1373 - accuracy: 0.9618 - val_loss: 1.1053 - val_accuracy: 0.3308
Epoch 8/10
17/17 [=============

17/17 [==============================] - 53s 2s/step - loss: 0.6462 - accuracy: 0.7855 - val_loss: 1.1108 - val_accuracy: 0.3308
Epoch 2/10
17/17 [==============================] - 33s 2s/step - loss: 0.5363 - accuracy: 0.7889 - val_loss: 1.1175 - val_accuracy: 0.3308
Epoch 3/10
17/17 [==============================] - 32s 2s/step - loss: 0.3077 - accuracy: 0.8795 - val_loss: 1.1525 - val_accuracy: 0.3308
Epoch 4/10
17/17 [==============================] - 32s 2s/step - loss: 0.2065 - accuracy: 0.9365 - val_loss: 1.1380 - val_accuracy: 0.3308
Epoch 5/10
17/17 [==============================] - 32s 2s/step - loss: 0.1625 - accuracy: 0.9390 - val_loss: 1.2735 - val_accuracy: 0.3308
Epoch 6/10
17/17 [==============================] - 32s 2s/step - loss: 0.0984 - accuracy: 0.9745 - val_loss: 1.2398 - val_accuracy: 0.3308
Epoch 7/10
17/17 [==============================] - 32s 2s/step - loss: 0.1296 - accuracy: 0.9578 - val_loss: 1.1772 - val_accuracy: 0.3308
Epoch 8/10
17/17 [=============

In [13]:
prune_model(model_path='beans_models/EfficentNetB0_beans_model.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1], 
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.5)
prune_model(model_path='beans_models/EfficentNetB0_beans_model.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1], 
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.75)
prune_model(model_path='beans_models/EfficentNetB0_beans_model.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1], 
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.9)

Epoch 1/10


ResourceExhaustedError:  OOM when allocating tensor with shape[64,32,32,480] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node model_1/prune_low_magnitude_block4c_activation/mul-0-1-TransposeNCHWToNHWC-LayoutOptimizer}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_548175]

Function call stack:
train_function


## 2. Weight cluster base models

In [14]:
weight_cluster_model(model_path='flowers_models/MobileNetV2_flowers_model.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models/MobileNetV2_flowers_model.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
41/41 [==============================] - 38s 572ms/step - loss: 0.2422 - accuracy: 0.9275 - val_loss: 1.3989 - val_accuracy: 0.7005
Epoch 2/10
41/41 [==============================] - 39s 522ms/step - loss: 0.0117 - accuracy: 0.9978 - val_loss: 1.6330 - val_accuracy: 0.6933
Epoch 3/10
41/41 [==============================] - 32s 539ms/step - loss: 0.0145 - accuracy: 0.9946 - val_loss: 1.7333 - val_accuracy: 0.6733
Epoch 4/10
41/41 [==============================] - 30s 517ms/step - loss: 0.0027 - accuracy: 0.9988 - val_loss: 1.9644 - val_accuracy: 0.6860
Epoch 5/10
41/41 [==============================] - 40s 531ms/step - loss: 0.0052 - accuracy: 0.9973 - val_loss: 1.5252 - val_accuracy: 0.7042
Epoch 6/10
41/41 [==============================] - 36s 544ms/step - loss: 0.0199 - accuracy: 0.9924 - val_loss: 2.6876 - val_accuracy: 0.6316
Epoch 7/10
41/41 [==============================] - 39s 528ms/step - loss: 0.0790 - accuracy: 0.9706 - val_loss: 1.6436 - val_accuracy: 0.6878

In [15]:
weight_cluster_model(model_path='flowers_models/EfficentNetB0_flowers_model.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models/EfficentNetB0_flowers_model.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
41/41 [==============================] - 51s 755ms/step - loss: 0.0165 - accuracy: 0.9950 - val_loss: 1.6021 - val_accuracy: 0.6933
Epoch 2/10
41/41 [==============================] - 41s 739ms/step - loss: 0.0118 - accuracy: 0.9960 - val_loss: 1.8646 - val_accuracy: 0.6897
Epoch 3/10
41/41 [==============================] - 46s 721ms/step - loss: 0.0060 - accuracy: 0.9978 - val_loss: 1.8520 - val_accuracy: 0.6933
Epoch 4/10
41/41 [==============================] - 47s 732ms/step - loss: 7.6542e-04 - accuracy: 0.9999 - val_loss: 1.8964 - val_accuracy: 0.6951
Epoch 5/10
41/41 [==============================] - 43s 697ms/step - loss: 6.3385e-04 - accuracy: 0.9993 - val_loss: 1.9094 - val_accuracy: 0.6951
Epoch 6/10
41/41 [==============================] - 42s 744ms/step - loss: 4.1009e-04 - accuracy: 0.9995 - val_loss: 1.9348 - val_accuracy: 0.6969
Epoch 7/10
41/41 [==============================] - 45s 710ms/step - loss: 0.0014 - accuracy: 0.9994 - val_loss: 1.9459 - val_accu

In [16]:
weight_cluster_model(model_path='beans_models/MobileNetV2_beans_model.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models/MobileNetV2_beans_model.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
17/17 [==============================] - 39s 2s/step - loss: 5.9402 - accuracy: 0.5883 - val_loss: 0.8820 - val_accuracy: 0.5489
Epoch 2/10
17/17 [==============================] - 34s 2s/step - loss: 0.8479 - accuracy: 0.6229 - val_loss: 0.7968 - val_accuracy: 0.6165
Epoch 3/10
17/17 [==============================] - 34s 2s/step - loss: 0.7125 - accuracy: 0.6890 - val_loss: 0.6677 - val_accuracy: 0.6692
Epoch 4/10
17/17 [==============================] - 34s 2s/step - loss: 0.6054 - accuracy: 0.7477 - val_loss: 0.6110 - val_accuracy: 0.7068
Epoch 5/10
17/17 [==============================] - 34s 2s/step - loss: 0.4822 - accuracy: 0.8004 - val_loss: 0.7550 - val_accuracy: 0.6992
Epoch 6/10
17/17 [==============================] - 34s 2s/step - loss: 0.3038 - accuracy: 0.8860 - val_loss: 0.7577 - val_accuracy: 0.7744
Epoch 7/10
17/17 [==============================] - 34s 2s/step - loss: 0.1932 - accuracy: 0.9362 - val_loss: 0.8160 - val_accuracy: 0.7368
Epoch 8/10
17/17 [==

In [17]:
weight_cluster_model(model_path='beans_models/EfficentNetB0_beans_model.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models/EfficentNetB0_beans_model.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10


ResourceExhaustedError:  OOM when allocating tensor with shape[64,32,32,480] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node model_1/cluster_block4c_activation/mul-0-1-TransposeNCHWToNHWC-LayoutOptimizer}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_746724]

Function call stack:
train_function


## 3. weight cluster prunned models

In [18]:
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_ConstantSparsity50.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_ConstantSparsity50.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
41/41 [==============================] - 40s 563ms/step - loss: 3.4468 - accuracy: 0.6025 - val_loss: 1.1391 - val_accuracy: 0.6570
Epoch 2/10
41/41 [==============================] - 31s 530ms/step - loss: 0.1773 - accuracy: 0.9416 - val_loss: 1.4946 - val_accuracy: 0.6860
Epoch 3/10
41/41 [==============================] - 37s 560ms/step - loss: 0.0394 - accuracy: 0.9860 - val_loss: 1.4401 - val_accuracy: 0.7169
Epoch 4/10
41/41 [==============================] - 40s 530ms/step - loss: 0.0047 - accuracy: 0.9988 - val_loss: 1.6013 - val_accuracy: 0.7205
Epoch 5/10
41/41 [==============================] - 36s 535ms/step - loss: 0.0023 - accuracy: 0.9998 - val_loss: 1.6630 - val_accuracy: 0.7114
Epoch 6/10
41/41 [==============================] - 37s 533ms/step - loss: 0.0039 - accuracy: 0.9990 - val_loss: 1.7817 - val_accuracy: 0.7151
Epoch 7/10
41/41 [==============================] - 36s 551ms/step - loss: 0.0026 - accuracy: 0.9989 - val_loss: 1.9477 - val_accuracy: 0.6933

Epoch 1/10
41/41 [==============================] - 36s 550ms/step - loss: 3.8840 - accuracy: 0.5132 - val_loss: 1.0528 - val_accuracy: 0.6207
Epoch 2/10
41/41 [==============================] - 37s 545ms/step - loss: 0.5530 - accuracy: 0.8009 - val_loss: 1.1481 - val_accuracy: 0.6860
Epoch 3/10
41/41 [==============================] - 36s 533ms/step - loss: 0.1340 - accuracy: 0.9475 - val_loss: 1.4633 - val_accuracy: 0.7024
Epoch 4/10
41/41 [==============================] - 39s 528ms/step - loss: 0.0177 - accuracy: 0.9983 - val_loss: 1.3589 - val_accuracy: 0.7132
Epoch 5/10
41/41 [==============================] - 37s 540ms/step - loss: 0.0062 - accuracy: 0.9985 - val_loss: 1.4632 - val_accuracy: 0.7223
Epoch 6/10
41/41 [==============================] - 39s 530ms/step - loss: 0.0017 - accuracy: 0.9999 - val_loss: 1.5491 - val_accuracy: 0.7187
Epoch 7/10
41/41 [==============================] - 36s 532ms/step - loss: 0.0024 - accuracy: 0.9993 - val_loss: 1.5801 - val_accuracy: 0.7187

In [19]:
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_PolynomialDecay50.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_PolynomialDecay50.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
41/41 [==============================] - 34s 560ms/step - loss: 3.6074 - accuracy: 0.5775 - val_loss: 1.0917 - val_accuracy: 0.6370
Epoch 2/10
41/41 [==============================] - 34s 541ms/step - loss: 0.5100 - accuracy: 0.8079 - val_loss: 1.2421 - val_accuracy: 0.6806
Epoch 3/10
41/41 [==============================] - 35s 553ms/step - loss: 0.0607 - accuracy: 0.9780 - val_loss: 1.3934 - val_accuracy: 0.6969
Epoch 4/10
41/41 [==============================] - 31s 522ms/step - loss: 0.0169 - accuracy: 0.9974 - val_loss: 1.4361 - val_accuracy: 0.7296
Epoch 5/10
41/41 [==============================] - 33s 529ms/step - loss: 0.0046 - accuracy: 0.9997 - val_loss: 1.5807 - val_accuracy: 0.7278
Epoch 6/10
41/41 [==============================] - 38s 543ms/step - loss: 9.5194e-04 - accuracy: 1.0000 - val_loss: 1.6546 - val_accuracy: 0.7296
Epoch 7/10
41/41 [==============================] - 40s 541ms/step - loss: 0.0012 - accuracy: 0.9999 - val_loss: 1.6129 - val_accuracy: 0.

Epoch 1/10
41/41 [==============================] - 36s 539ms/step - loss: 3.7807 - accuracy: 0.5037 - val_loss: 1.0383 - val_accuracy: 0.6298
Epoch 2/10
41/41 [==============================] - 39s 535ms/step - loss: 0.5721 - accuracy: 0.7806 - val_loss: 1.1069 - val_accuracy: 0.6770
Epoch 3/10
41/41 [==============================] - 39s 531ms/step - loss: 0.1144 - accuracy: 0.9599 - val_loss: 1.3495 - val_accuracy: 0.7005
Epoch 4/10
41/41 [==============================] - 39s 540ms/step - loss: 0.0248 - accuracy: 0.9954 - val_loss: 1.6829 - val_accuracy: 0.7024
Epoch 5/10
41/41 [==============================] - 38s 529ms/step - loss: 0.0202 - accuracy: 0.9947 - val_loss: 1.7478 - val_accuracy: 0.7005
Epoch 6/10
41/41 [==============================] - 36s 541ms/step - loss: 0.0096 - accuracy: 0.9970 - val_loss: 1.7297 - val_accuracy: 0.7042
Epoch 7/10
41/41 [==============================] - 35s 535ms/step - loss: 0.0036 - accuracy: 0.9993 - val_loss: 1.7907 - val_accuracy: 0.7187

In [20]:
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_ConstantSparsity75.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_ConstantSparsity75.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
41/41 [==============================] - 44s 542ms/step - loss: 1.6680 - accuracy: 0.5581 - val_loss: 0.8394 - val_accuracy: 0.7078
Epoch 2/10
41/41 [==============================] - 29s 531ms/step - loss: 0.2244 - accuracy: 0.9278 - val_loss: 1.0915 - val_accuracy: 0.7005
Epoch 3/10
41/41 [==============================] - 39s 524ms/step - loss: 0.1318 - accuracy: 0.9560 - val_loss: 1.5383 - val_accuracy: 0.6860
Epoch 4/10
41/41 [==============================] - 32s 549ms/step - loss: 0.0438 - accuracy: 0.9855 - val_loss: 1.4699 - val_accuracy: 0.7151
Epoch 5/10
41/41 [==============================] - 35s 521ms/step - loss: 0.0577 - accuracy: 0.9802 - val_loss: 1.3685 - val_accuracy: 0.6987
Epoch 6/10
41/41 [==============================] - 38s 509ms/step - loss: 0.0679 - accuracy: 0.9762 - val_loss: 1.6055 - val_accuracy: 0.6842
Epoch 7/10
41/41 [==============================] - 32s 518ms/step - loss: 0.0913 - accuracy: 0.9730 - val_loss: 1.5071 - val_accuracy: 0.7060

Epoch 1/10
41/41 [==============================] - 46s 564ms/step - loss: 1.3436 - accuracy: 0.6323 - val_loss: 0.9331 - val_accuracy: 0.6915
Epoch 2/10
41/41 [==============================] - 39s 516ms/step - loss: 0.1829 - accuracy: 0.9357 - val_loss: 1.0338 - val_accuracy: 0.6951
Epoch 3/10
41/41 [==============================] - 40s 533ms/step - loss: 0.0529 - accuracy: 0.9858 - val_loss: 1.6777 - val_accuracy: 0.6661
Epoch 4/10
41/41 [==============================] - 37s 515ms/step - loss: 0.0792 - accuracy: 0.9711 - val_loss: 1.3216 - val_accuracy: 0.7024
Epoch 5/10
41/41 [==============================] - 38s 514ms/step - loss: 0.0597 - accuracy: 0.9784 - val_loss: 1.4651 - val_accuracy: 0.6987
Epoch 6/10
41/41 [==============================] - 32s 535ms/step - loss: 0.0393 - accuracy: 0.9855 - val_loss: 1.5944 - val_accuracy: 0.6897
Epoch 7/10
41/41 [==============================] - 35s 502ms/step - loss: 0.0385 - accuracy: 0.9866 - val_loss: 1.6307 - val_accuracy: 0.6679

In [21]:
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_PolynomialDecay75.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_PolynomialDecay75.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
41/41 [==============================] - 43s 539ms/step - loss: 1.3461 - accuracy: 0.6531 - val_loss: 0.9117 - val_accuracy: 0.6806
Epoch 2/10
41/41 [==============================] - 39s 536ms/step - loss: 0.2257 - accuracy: 0.9209 - val_loss: 1.5061 - val_accuracy: 0.6751
Epoch 3/10
41/41 [==============================] - 39s 526ms/step - loss: 0.1282 - accuracy: 0.9555 - val_loss: 1.5870 - val_accuracy: 0.7187
Epoch 4/10
41/41 [==============================] - 34s 549ms/step - loss: 0.0430 - accuracy: 0.9872 - val_loss: 1.7722 - val_accuracy: 0.6987
Epoch 5/10
41/41 [==============================] - 38s 532ms/step - loss: 0.0664 - accuracy: 0.9795 - val_loss: 1.5529 - val_accuracy: 0.7005
Epoch 6/10
41/41 [==============================] - 27s 517ms/step - loss: 0.0569 - accuracy: 0.9774 - val_loss: 1.7572 - val_accuracy: 0.7151
Epoch 7/10
41/41 [==============================] - 40s 538ms/step - loss: 0.0599 - accuracy: 0.9769 - val_loss: 1.6627 - val_accuracy: 0.6788

Epoch 1/10
41/41 [==============================] - 45s 548ms/step - loss: 1.4008 - accuracy: 0.6421 - val_loss: 1.0369 - val_accuracy: 0.6951
Epoch 2/10
41/41 [==============================] - 33s 525ms/step - loss: 0.2066 - accuracy: 0.9295 - val_loss: 1.0783 - val_accuracy: 0.6951
Epoch 3/10
41/41 [==============================] - 35s 539ms/step - loss: 0.1070 - accuracy: 0.9628 - val_loss: 1.4481 - val_accuracy: 0.6733
Epoch 4/10
41/41 [==============================] - 37s 521ms/step - loss: 0.0450 - accuracy: 0.9885 - val_loss: 1.9170 - val_accuracy: 0.6425
Epoch 5/10
41/41 [==============================] - 31s 534ms/step - loss: 0.0735 - accuracy: 0.9730 - val_loss: 1.7052 - val_accuracy: 0.6860
Epoch 6/10
41/41 [==============================] - 39s 533ms/step - loss: 0.0856 - accuracy: 0.9724 - val_loss: 1.4337 - val_accuracy: 0.6897
Epoch 7/10
41/41 [==============================] - 37s 517ms/step - loss: 0.0904 - accuracy: 0.9731 - val_loss: 1.4747 - val_accuracy: 0.6842

In [22]:
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_ConstantSparsity90.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_ConstantSparsity90.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
41/41 [==============================] - 44s 530ms/step - loss: 2.3170 - accuracy: 0.2446 - val_loss: 1.5701 - val_accuracy: 0.2577
Epoch 2/10
41/41 [==============================] - 38s 529ms/step - loss: 1.4945 - accuracy: 0.3264 - val_loss: 1.2094 - val_accuracy: 0.4809
Epoch 3/10
41/41 [==============================] - 32s 526ms/step - loss: 1.0814 - accuracy: 0.5276 - val_loss: 1.0143 - val_accuracy: 0.6007
Epoch 4/10
41/41 [==============================] - 37s 539ms/step - loss: 0.8931 - accuracy: 0.6649 - val_loss: 0.8992 - val_accuracy: 0.6733
Epoch 5/10
41/41 [==============================] - 40s 550ms/step - loss: 0.7154 - accuracy: 0.7311 - val_loss: 0.9913 - val_accuracy: 0.6642
Epoch 6/10
41/41 [==============================] - 28s 529ms/step - loss: 0.5163 - accuracy: 0.8072 - val_loss: 0.9109 - val_accuracy: 0.6770
Epoch 7/10
41/41 [==============================] - 39s 536ms/step - loss: 0.4150 - accuracy: 0.8403 - val_loss: 0.9682 - val_accuracy: 0.6824

Epoch 1/10
41/41 [==============================] - 38s 549ms/step - loss: 2.3525 - accuracy: 0.2436 - val_loss: 1.4935 - val_accuracy: 0.3085
Epoch 2/10
41/41 [==============================] - 40s 565ms/step - loss: 1.4036 - accuracy: 0.3542 - val_loss: 1.2694 - val_accuracy: 0.4864
Epoch 3/10
41/41 [==============================] - 36s 541ms/step - loss: 1.0584 - accuracy: 0.5784 - val_loss: 0.8362 - val_accuracy: 0.6788
Epoch 4/10
41/41 [==============================] - 38s 526ms/step - loss: 0.7894 - accuracy: 0.6802 - val_loss: 0.9309 - val_accuracy: 0.6279
Epoch 5/10
41/41 [==============================] - 39s 532ms/step - loss: 0.6373 - accuracy: 0.7420 - val_loss: 0.8501 - val_accuracy: 0.7078
Epoch 6/10
41/41 [==============================] - 30s 537ms/step - loss: 0.4910 - accuracy: 0.8214 - val_loss: 0.8350 - val_accuracy: 0.6842
Epoch 7/10
41/41 [==============================] - 38s 555ms/step - loss: 0.3323 - accuracy: 0.8871 - val_loss: 1.0169 - val_accuracy: 0.6842

In [23]:
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_PolynomialDecay90.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_PolynomialDecay90.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
41/41 [==============================] - 39s 545ms/step - loss: 2.2986 - accuracy: 0.2471 - val_loss: 1.5901 - val_accuracy: 0.3267
Epoch 2/10
41/41 [==============================] - 32s 544ms/step - loss: 1.5723 - accuracy: 0.2800 - val_loss: 1.4042 - val_accuracy: 0.4138
Epoch 3/10
41/41 [==============================] - 29s 534ms/step - loss: 1.2855 - accuracy: 0.4558 - val_loss: 1.0174 - val_accuracy: 0.6134
Epoch 4/10
41/41 [==============================] - 40s 547ms/step - loss: 0.8450 - accuracy: 0.6816 - val_loss: 0.7900 - val_accuracy: 0.6860
Epoch 5/10
41/41 [==============================] - 37s 561ms/step - loss: 0.6957 - accuracy: 0.7244 - val_loss: 1.1188 - val_accuracy: 0.5880
Epoch 6/10
41/41 [==============================] - 40s 547ms/step - loss: 0.6307 - accuracy: 0.7653 - val_loss: 0.8620 - val_accuracy: 0.7187
Epoch 7/10
41/41 [==============================] - 38s 545ms/step - loss: 0.5576 - accuracy: 0.8035 - val_loss: 0.9451 - val_accuracy: 0.7005

Epoch 1/10
41/41 [==============================] - 39s 577ms/step - loss: 2.3900 - accuracy: 0.2337 - val_loss: 1.5754 - val_accuracy: 0.2432
Epoch 2/10
41/41 [==============================] - 40s 535ms/step - loss: 1.5334 - accuracy: 0.3083 - val_loss: 1.1458 - val_accuracy: 0.5172
Epoch 3/10
41/41 [==============================] - 32s 559ms/step - loss: 1.1926 - accuracy: 0.5319 - val_loss: 0.9183 - val_accuracy: 0.6171
Epoch 4/10
41/41 [==============================] - 32s 520ms/step - loss: 0.8123 - accuracy: 0.6874 - val_loss: 0.9012 - val_accuracy: 0.6189
Epoch 5/10
41/41 [==============================] - 40s 536ms/step - loss: 0.6468 - accuracy: 0.7486 - val_loss: 0.8851 - val_accuracy: 0.6515
Epoch 6/10
41/41 [==============================] - 34s 526ms/step - loss: 0.6284 - accuracy: 0.7603 - val_loss: 1.0130 - val_accuracy: 0.6897
Epoch 7/10
41/41 [==============================] - 30s 543ms/step - loss: 0.4276 - accuracy: 0.8452 - val_loss: 0.9481 - val_accuracy: 0.6969

In [24]:
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_ConstantSparsity50.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_ConstantSparsity50.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
41/41 [==============================] - 51s 734ms/step - loss: 0.0298 - accuracy: 0.9880 - val_loss: 2.2341 - val_accuracy: 0.6770
Epoch 2/10
41/41 [==============================] - 38s 711ms/step - loss: 0.0153 - accuracy: 0.9930 - val_loss: 2.4373 - val_accuracy: 0.6624
Epoch 3/10
41/41 [==============================] - 41s 697ms/step - loss: 0.0017 - accuracy: 0.9990 - val_loss: 2.5389 - val_accuracy: 0.6860
Epoch 4/10
41/41 [==============================] - 42s 723ms/step - loss: 8.0469e-04 - accuracy: 0.9998 - val_loss: 2.9173 - val_accuracy: 0.6770
Epoch 5/10
41/41 [==============================] - 43s 734ms/step - loss: 0.0271 - accuracy: 0.9877 - val_loss: 1.9926 - val_accuracy: 0.6751
Epoch 6/10
41/41 [==============================] - 40s 716ms/step - loss: 0.0318 - accuracy: 0.9917 - val_loss: 1.6079 - val_accuracy: 0.7005
Epoch 7/10
41/41 [==============================] - 43s 718ms/step - loss: 0.0030 - accuracy: 0.9982 - val_loss: 1.9472 - val_accuracy: 0.

Epoch 1/10
41/41 [==============================] - 52s 732ms/step - loss: 0.0414 - accuracy: 0.9853 - val_loss: 1.4776 - val_accuracy: 0.6951
Epoch 2/10
41/41 [==============================] - 44s 718ms/step - loss: 0.0120 - accuracy: 0.9954 - val_loss: 2.0492 - val_accuracy: 0.6788
Epoch 3/10
41/41 [==============================] - 43s 746ms/step - loss: 0.0105 - accuracy: 0.9948 - val_loss: 1.8969 - val_accuracy: 0.6915
Epoch 4/10
41/41 [==============================] - 40s 720ms/step - loss: 7.1802e-04 - accuracy: 0.9997 - val_loss: 1.9054 - val_accuracy: 0.7096
Epoch 5/10
41/41 [==============================] - 43s 724ms/step - loss: 6.2288e-04 - accuracy: 0.9998 - val_loss: 1.8869 - val_accuracy: 0.7060
Epoch 6/10
41/41 [==============================] - 43s 719ms/step - loss: 4.9915e-04 - accuracy: 0.9998 - val_loss: 1.8641 - val_accuracy: 0.7151
Epoch 7/10
41/41 [==============================] - 46s 743ms/step - loss: 3.5634e-04 - accuracy: 0.9996 - val_loss: 1.8790 - val_

In [25]:
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_PolynomialDecay50.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_PolynomialDecay50.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
41/41 [==============================] - 56s 734ms/step - loss: 0.0376 - accuracy: 0.9886 - val_loss: 1.9998 - val_accuracy: 0.6987
Epoch 2/10
41/41 [==============================] - 46s 733ms/step - loss: 0.0187 - accuracy: 0.9949 - val_loss: 2.2093 - val_accuracy: 0.6624
Epoch 3/10
41/41 [==============================] - 40s 714ms/step - loss: 0.0219 - accuracy: 0.9926 - val_loss: 2.2961 - val_accuracy: 0.6878
Epoch 4/10
41/41 [==============================] - 41s 704ms/step - loss: 0.0050 - accuracy: 0.9988 - val_loss: 2.3901 - val_accuracy: 0.6824
Epoch 5/10
41/41 [==============================] - 38s 700ms/step - loss: 0.0150 - accuracy: 0.9944 - val_loss: 1.9411 - val_accuracy: 0.7005
Epoch 6/10
41/41 [==============================] - 42s 724ms/step - loss: 0.0031 - accuracy: 0.9983 - val_loss: 2.2021 - val_accuracy: 0.7060
Epoch 7/10
41/41 [==============================] - 43s 705ms/step - loss: 0.0031 - accuracy: 0.9986 - val_loss: 2.0725 - val_accuracy: 0.6860

Epoch 1/10
41/41 [==============================] - 47s 722ms/step - loss: 0.0679 - accuracy: 0.9731 - val_loss: 1.5315 - val_accuracy: 0.7078
Epoch 2/10
41/41 [==============================] - 47s 721ms/step - loss: 0.0127 - accuracy: 0.9955 - val_loss: 2.2291 - val_accuracy: 0.6788
Epoch 3/10
41/41 [==============================] - 47s 711ms/step - loss: 0.0082 - accuracy: 0.9984 - val_loss: 1.7689 - val_accuracy: 0.7078
Epoch 4/10
41/41 [==============================] - 48s 732ms/step - loss: 0.0015 - accuracy: 0.9989 - val_loss: 1.8582 - val_accuracy: 0.7096
Epoch 5/10
41/41 [==============================] - 47s 710ms/step - loss: 7.2017e-04 - accuracy: 0.9998 - val_loss: 1.8991 - val_accuracy: 0.7132
Epoch 6/10
41/41 [==============================] - 46s 701ms/step - loss: 0.0011 - accuracy: 0.9998 - val_loss: 1.8809 - val_accuracy: 0.7187
Epoch 7/10
41/41 [==============================] - 43s 722ms/step - loss: 0.0011 - accuracy: 0.9988 - val_loss: 1.8881 - val_accuracy: 0.

In [26]:
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_ConstantSparsity75.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_ConstantSparsity75.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
41/41 [==============================] - 56s 744ms/step - loss: 0.1740 - accuracy: 0.9414 - val_loss: 1.7632 - val_accuracy: 0.6407
Epoch 2/10
41/41 [==============================] - 38s 705ms/step - loss: 0.0056 - accuracy: 0.9992 - val_loss: 2.1240 - val_accuracy: 0.6661
Epoch 3/10
41/41 [==============================] - 37s 714ms/step - loss: 0.0065 - accuracy: 0.9975 - val_loss: 2.2310 - val_accuracy: 0.6370
Epoch 4/10
41/41 [==============================] - 47s 715ms/step - loss: 0.0258 - accuracy: 0.9901 - val_loss: 2.1459 - val_accuracy: 0.6461
Epoch 5/10
41/41 [==============================] - 46s 724ms/step - loss: 0.0079 - accuracy: 0.9961 - val_loss: 2.2262 - val_accuracy: 0.6152
Epoch 6/10
41/41 [==============================] - 44s 708ms/step - loss: 0.0203 - accuracy: 0.9930 - val_loss: 2.2004 - val_accuracy: 0.6497
Epoch 7/10
41/41 [==============================] - 42s 707ms/step - loss: 0.0125 - accuracy: 0.9951 - val_loss: 2.2734 - val_accuracy: 0.6606

Epoch 1/10
41/41 [==============================] - 55s 737ms/step - loss: 0.1205 - accuracy: 0.9604 - val_loss: 1.9269 - val_accuracy: 0.6370
Epoch 2/10
41/41 [==============================] - 34s 692ms/step - loss: 0.0133 - accuracy: 0.9954 - val_loss: 2.2052 - val_accuracy: 0.6171
Epoch 3/10
41/41 [==============================] - 37s 722ms/step - loss: 0.0934 - accuracy: 0.9820 - val_loss: 1.8624 - val_accuracy: 0.6751
Epoch 4/10
41/41 [==============================] - 47s 709ms/step - loss: 0.0028 - accuracy: 0.9997 - val_loss: 2.2102 - val_accuracy: 0.6642
Epoch 5/10
41/41 [==============================] - 41s 723ms/step - loss: 0.0011 - accuracy: 0.9998 - val_loss: 2.1053 - val_accuracy: 0.6842
Epoch 6/10
41/41 [==============================] - 39s 725ms/step - loss: 0.0017 - accuracy: 0.9995 - val_loss: 2.2324 - val_accuracy: 0.6697
Epoch 7/10
41/41 [==============================] - 46s 726ms/step - loss: 9.1571e-04 - accuracy: 0.9998 - val_loss: 2.2714 - val_accuracy: 0.

In [27]:
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_PolynomialDecay75.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_PolynomialDecay75.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
41/41 [==============================] - 52s 756ms/step - loss: 0.1975 - accuracy: 0.9391 - val_loss: 1.5283 - val_accuracy: 0.6570
Epoch 2/10
41/41 [==============================] - 41s 698ms/step - loss: 0.0081 - accuracy: 0.9969 - val_loss: 1.9614 - val_accuracy: 0.6679
Epoch 3/10
41/41 [==============================] - 46s 694ms/step - loss: 0.0043 - accuracy: 0.9989 - val_loss: 1.8550 - val_accuracy: 0.6715
Epoch 4/10
41/41 [==============================] - 43s 697ms/step - loss: 8.3650e-04 - accuracy: 0.9997 - val_loss: 1.9563 - val_accuracy: 0.6770
Epoch 5/10
41/41 [==============================] - 45s 699ms/step - loss: 6.5315e-04 - accuracy: 0.9998 - val_loss: 2.0094 - val_accuracy: 0.6733
Epoch 6/10
41/41 [==============================] - 42s 703ms/step - loss: 0.0012 - accuracy: 0.9994 - val_loss: 2.0686 - val_accuracy: 0.6697
Epoch 7/10
41/41 [==============================] - 46s 697ms/step - loss: 6.2421e-04 - accuracy: 0.9997 - val_loss: 2.1102 - val_accu

Epoch 1/10
41/41 [==============================] - 57s 742ms/step - loss: 0.1822 - accuracy: 0.9467 - val_loss: 1.5955 - val_accuracy: 0.6479
Epoch 2/10
41/41 [==============================] - 43s 716ms/step - loss: 0.0111 - accuracy: 0.9973 - val_loss: 2.0807 - val_accuracy: 0.6860
Epoch 3/10
41/41 [==============================] - 47s 719ms/step - loss: 0.0013 - accuracy: 0.9999 - val_loss: 2.1446 - val_accuracy: 0.6697
Epoch 4/10
41/41 [==============================] - 41s 724ms/step - loss: 8.4011e-04 - accuracy: 0.9999 - val_loss: 2.2147 - val_accuracy: 0.6751
Epoch 5/10
41/41 [==============================] - 39s 717ms/step - loss: 3.9544e-04 - accuracy: 1.0000 - val_loss: 2.2809 - val_accuracy: 0.6715
Epoch 6/10
41/41 [==============================] - 44s 726ms/step - loss: 2.1791e-04 - accuracy: 0.9999 - val_loss: 2.3209 - val_accuracy: 0.6733
Epoch 7/10
41/41 [==============================] - 47s 713ms/step - loss: 0.0012 - accuracy: 0.9995 - val_loss: 2.3672 - val_accu

In [28]:
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_ConstantSparsity90.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_ConstantSparsity90.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
41/41 [==============================] - 54s 750ms/step - loss: 0.1746 - accuracy: 0.9432 - val_loss: 1.7766 - val_accuracy: 0.6534
Epoch 2/10
41/41 [==============================] - 35s 712ms/step - loss: 0.0201 - accuracy: 0.9950 - val_loss: 1.5539 - val_accuracy: 0.6715
Epoch 3/10
41/41 [==============================] - 45s 716ms/step - loss: 0.0069 - accuracy: 0.9993 - val_loss: 2.2180 - val_accuracy: 0.6515
Epoch 4/10
41/41 [==============================] - 47s 723ms/step - loss: 0.0076 - accuracy: 0.9993 - val_loss: 2.2429 - val_accuracy: 0.6443
Epoch 5/10
41/41 [==============================] - 39s 717ms/step - loss: 0.0074 - accuracy: 0.9973 - val_loss: 2.2695 - val_accuracy: 0.6606
Epoch 6/10
41/41 [==============================] - 43s 731ms/step - loss: 0.0106 - accuracy: 0.9959 - val_loss: 1.7107 - val_accuracy: 0.6388
Epoch 7/10
41/41 [==============================] - 42s 706ms/step - loss: 0.0274 - accuracy: 0.9883 - val_loss: 2.0609 - val_accuracy: 0.6334

Epoch 1/10
41/41 [==============================] - 52s 748ms/step - loss: 0.1755 - accuracy: 0.9479 - val_loss: 2.0997 - val_accuracy: 0.6770
Epoch 2/10
41/41 [==============================] - 47s 709ms/step - loss: 0.0384 - accuracy: 0.9886 - val_loss: 1.6296 - val_accuracy: 0.6951
Epoch 3/10
41/41 [==============================] - 46s 705ms/step - loss: 0.0132 - accuracy: 0.9980 - val_loss: 2.1853 - val_accuracy: 0.6588
Epoch 4/10
41/41 [==============================] - 40s 711ms/step - loss: 0.1288 - accuracy: 0.9543 - val_loss: 2.1699 - val_accuracy: 0.6534
Epoch 5/10
41/41 [==============================] - 40s 703ms/step - loss: 0.0109 - accuracy: 0.9984 - val_loss: 2.5558 - val_accuracy: 0.6025
Epoch 6/10
41/41 [==============================] - 36s 718ms/step - loss: 0.0215 - accuracy: 0.9919 - val_loss: 2.4469 - val_accuracy: 0.6624
Epoch 7/10
41/41 [==============================] - 47s 717ms/step - loss: 0.0076 - accuracy: 0.9979 - val_loss: 2.3896 - val_accuracy: 0.6515

In [29]:
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_PolynomialDecay90.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_PolynomialDecay90.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
41/41 [==============================] - 53s 750ms/step - loss: 0.2053 - accuracy: 0.9462 - val_loss: 2.0548 - val_accuracy: 0.6715
Epoch 2/10
41/41 [==============================] - 46s 698ms/step - loss: 0.0277 - accuracy: 0.9907 - val_loss: 1.5865 - val_accuracy: 0.6606
Epoch 3/10
41/41 [==============================] - 43s 715ms/step - loss: 0.0154 - accuracy: 0.9948 - val_loss: 2.7450 - val_accuracy: 0.6425
Epoch 4/10
41/41 [==============================] - 44s 717ms/step - loss: 0.0325 - accuracy: 0.9870 - val_loss: 1.8178 - val_accuracy: 0.6479
Epoch 5/10
41/41 [==============================] - 38s 712ms/step - loss: 0.0139 - accuracy: 0.9950 - val_loss: 2.1500 - val_accuracy: 0.6515
Epoch 6/10
41/41 [==============================] - 40s 704ms/step - loss: 0.0394 - accuracy: 0.9859 - val_loss: 2.2193 - val_accuracy: 0.6661
Epoch 7/10
41/41 [==============================] - 45s 696ms/step - loss: 0.0110 - accuracy: 0.9980 - val_loss: 1.9688 - val_accuracy: 0.6661

Epoch 1/10
41/41 [==============================] - 53s 748ms/step - loss: 0.2112 - accuracy: 0.9366 - val_loss: 2.0624 - val_accuracy: 0.6334
Epoch 2/10
41/41 [==============================] - 45s 717ms/step - loss: 0.0587 - accuracy: 0.9819 - val_loss: 1.8344 - val_accuracy: 0.6588
Epoch 3/10
41/41 [==============================] - 46s 707ms/step - loss: 0.0365 - accuracy: 0.9868 - val_loss: 2.0403 - val_accuracy: 0.6588
Epoch 4/10
41/41 [==============================] - 41s 699ms/step - loss: 0.0208 - accuracy: 0.9938 - val_loss: 1.7086 - val_accuracy: 0.6770
Epoch 5/10
41/41 [==============================] - 44s 722ms/step - loss: 0.0447 - accuracy: 0.9869 - val_loss: 2.0229 - val_accuracy: 0.6606
Epoch 6/10
41/41 [==============================] - 40s 703ms/step - loss: 0.0031 - accuracy: 0.9999 - val_loss: 2.1333 - val_accuracy: 0.6715
Epoch 7/10
41/41 [==============================] - 41s 712ms/step - loss: 0.0032 - accuracy: 0.9998 - val_loss: 1.8224 - val_accuracy: 0.6788

#### beans models

In [30]:
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity50.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity50.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
17/17 [==============================] - 39s 2s/step - loss: 11.7111 - accuracy: 0.3637 - val_loss: 1.5792 - val_accuracy: 0.3759
Epoch 2/10
17/17 [==============================] - 34s 2s/step - loss: 1.3878 - accuracy: 0.3591 - val_loss: 1.1111 - val_accuracy: 0.3308
Epoch 3/10
17/17 [==============================] - 34s 2s/step - loss: 1.1124 - accuracy: 0.3157 - val_loss: 1.1037 - val_accuracy: 0.3308
Epoch 4/10
17/17 [==============================] - 34s 2s/step - loss: 1.0996 - accuracy: 0.3424 - val_loss: 1.1018 - val_accuracy: 0.3308
Epoch 5/10
17/17 [==============================] - 34s 2s/step - loss: 1.1012 - accuracy: 0.3434 - val_loss: 1.1000 - val_accuracy: 0.3308
Epoch 6/10
17/17 [==============================] - 34s 2s/step - loss: 1.0995 - accuracy: 0.3327 - val_loss: 1.0995 - val_accuracy: 0.3308
Epoch 7/10
17/17 [==============================] - 34s 2s/step - loss: 1.1003 - accuracy: 0.3208 - val_loss: 1.0991 - val_accuracy: 0.3308
Epoch 8/10
17/17 [=

Epoch 1/10
17/17 [==============================] - 40s 2s/step - loss: 13.1270 - accuracy: 0.3340 - val_loss: 1.3130 - val_accuracy: 0.3308
Epoch 2/10
17/17 [==============================] - 34s 2s/step - loss: 1.2249 - accuracy: 0.3384 - val_loss: 1.1171 - val_accuracy: 0.3308
Epoch 3/10
17/17 [==============================] - 34s 2s/step - loss: 1.1131 - accuracy: 0.3263 - val_loss: 1.1005 - val_accuracy: 0.3308
Epoch 4/10
17/17 [==============================] - 35s 2s/step - loss: 1.0995 - accuracy: 0.3316 - val_loss: 1.0995 - val_accuracy: 0.3308
Epoch 5/10
17/17 [==============================] - 35s 2s/step - loss: 1.1000 - accuracy: 0.3272 - val_loss: 1.0993 - val_accuracy: 0.3308
Epoch 6/10
17/17 [==============================] - 34s 2s/step - loss: 1.0989 - accuracy: 0.3325 - val_loss: 1.0990 - val_accuracy: 0.3308
Epoch 7/10
17/17 [==============================] - 34s 2s/step - loss: 1.0989 - accuracy: 0.3210 - val_loss: 1.0989 - val_accuracy: 0.3308
Epoch 8/10
17/17 [=

In [31]:
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_PolynomialDecay50.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_PolynomialDecay50.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
17/17 [==============================] - 42s 2s/step - loss: 9.7134 - accuracy: 0.4126 - val_loss: 1.3204 - val_accuracy: 0.3308
Epoch 2/10
17/17 [==============================] - 34s 2s/step - loss: 1.2603 - accuracy: 0.3665 - val_loss: 1.1434 - val_accuracy: 0.3308
Epoch 3/10
17/17 [==============================] - 34s 2s/step - loss: 1.1240 - accuracy: 0.3378 - val_loss: 1.0923 - val_accuracy: 0.3383
Epoch 4/10
17/17 [==============================] - 35s 2s/step - loss: 1.0903 - accuracy: 0.3556 - val_loss: 1.0808 - val_accuracy: 0.4662
Epoch 5/10
17/17 [==============================] - 35s 2s/step - loss: 1.0868 - accuracy: 0.4335 - val_loss: 1.0679 - val_accuracy: 0.4436
Epoch 6/10
17/17 [==============================] - 35s 2s/step - loss: 1.0726 - accuracy: 0.4048 - val_loss: 1.0485 - val_accuracy: 0.4060
Epoch 7/10
17/17 [==============================] - 35s 2s/step - loss: 1.0668 - accuracy: 0.4020 - val_loss: 1.0235 - val_accuracy: 0.4135
Epoch 8/10
17/17 [==

Epoch 1/10
17/17 [==============================] - 39s 2s/step - loss: 10.5365 - accuracy: 0.4055 - val_loss: 1.3383 - val_accuracy: 0.3308
Epoch 2/10
17/17 [==============================] - 34s 2s/step - loss: 1.2495 - accuracy: 0.3316 - val_loss: 1.1165 - val_accuracy: 0.3308
Epoch 3/10
17/17 [==============================] - 34s 2s/step - loss: 1.1069 - accuracy: 0.3412 - val_loss: 1.1029 - val_accuracy: 0.3308
Epoch 4/10
17/17 [==============================] - 34s 2s/step - loss: 1.1001 - accuracy: 0.3407 - val_loss: 1.1013 - val_accuracy: 0.3308
Epoch 5/10
17/17 [==============================] - 34s 2s/step - loss: 1.1010 - accuracy: 0.3272 - val_loss: 1.1002 - val_accuracy: 0.3308
Epoch 6/10
17/17 [==============================] - 34s 2s/step - loss: 1.0985 - accuracy: 0.3362 - val_loss: 1.0992 - val_accuracy: 0.3308
Epoch 7/10
17/17 [==============================] - 34s 2s/step - loss: 1.0988 - accuracy: 0.3427 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 8/10
17/17 [=

In [32]:
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity75.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity75.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
17/17 [==============================] - 39s 2s/step - loss: 4.2068 - accuracy: 0.3074 - val_loss: 1.0969 - val_accuracy: 0.3308
Epoch 2/10
17/17 [==============================] - 34s 2s/step - loss: 1.0968 - accuracy: 0.3527 - val_loss: 1.0959 - val_accuracy: 0.4060
Epoch 3/10
17/17 [==============================] - 34s 2s/step - loss: 1.0971 - accuracy: 0.3674 - val_loss: 1.0902 - val_accuracy: 0.3459
Epoch 4/10
17/17 [==============================] - 33s 2s/step - loss: 1.0866 - accuracy: 0.3838 - val_loss: 1.0625 - val_accuracy: 0.3835
Epoch 5/10
17/17 [==============================] - 34s 2s/step - loss: 1.0599 - accuracy: 0.4243 - val_loss: 1.0497 - val_accuracy: 0.4060
Epoch 6/10
17/17 [==============================] - 33s 2s/step - loss: 1.0665 - accuracy: 0.3822 - val_loss: 1.0255 - val_accuracy: 0.5038
Epoch 7/10
17/17 [==============================] - 34s 2s/step - loss: 1.0169 - accuracy: 0.4973 - val_loss: 1.0379 - val_accuracy: 0.4812
Epoch 8/10
17/17 [==

Epoch 1/10
17/17 [==============================] - 41s 2s/step - loss: 4.8890 - accuracy: 0.3156 - val_loss: 1.0974 - val_accuracy: 0.3308
Epoch 2/10
17/17 [==============================] - 35s 2s/step - loss: 1.0929 - accuracy: 0.3459 - val_loss: 1.0740 - val_accuracy: 0.4436
Epoch 3/10
17/17 [==============================] - 35s 2s/step - loss: 1.0822 - accuracy: 0.3996 - val_loss: 1.0553 - val_accuracy: 0.4211
Epoch 4/10
17/17 [==============================] - 35s 2s/step - loss: 1.0558 - accuracy: 0.4298 - val_loss: 1.0318 - val_accuracy: 0.4361
Epoch 5/10
17/17 [==============================] - 35s 2s/step - loss: 1.0515 - accuracy: 0.3928 - val_loss: 0.9636 - val_accuracy: 0.4962
Epoch 6/10
17/17 [==============================] - 35s 2s/step - loss: 1.0084 - accuracy: 0.4803 - val_loss: 1.0251 - val_accuracy: 0.5113
Epoch 7/10
17/17 [==============================] - 35s 2s/step - loss: 1.0326 - accuracy: 0.4889 - val_loss: 1.0038 - val_accuracy: 0.4887
Epoch 8/10
17/17 [==

In [33]:
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_PolynomialDecay75.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_PolynomialDecay75.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
17/17 [==============================] - 43s 2s/step - loss: 3.9838 - accuracy: 0.3164 - val_loss: 1.0993 - val_accuracy: 0.3083
Epoch 2/10
17/17 [==============================] - 35s 2s/step - loss: 1.1014 - accuracy: 0.3340 - val_loss: 1.0987 - val_accuracy: 0.3308
Epoch 3/10
17/17 [==============================] - 35s 2s/step - loss: 1.0983 - accuracy: 0.3324 - val_loss: 1.0970 - val_accuracy: 0.3308
Epoch 4/10
17/17 [==============================] - 35s 2s/step - loss: 1.0970 - accuracy: 0.3354 - val_loss: 1.0951 - val_accuracy: 0.3308
Epoch 5/10
17/17 [==============================] - 35s 2s/step - loss: 1.0949 - accuracy: 0.3456 - val_loss: 1.0886 - val_accuracy: 0.3609
Epoch 6/10
17/17 [==============================] - 35s 2s/step - loss: 1.0867 - accuracy: 0.3453 - val_loss: 1.0589 - val_accuracy: 0.4211
Epoch 7/10
17/17 [==============================] - 34s 2s/step - loss: 1.0641 - accuracy: 0.4090 - val_loss: 1.0189 - val_accuracy: 0.4511
Epoch 8/10
17/17 [==

Epoch 1/10
17/17 [==============================] - 39s 2s/step - loss: 4.7372 - accuracy: 0.3109 - val_loss: 1.1014 - val_accuracy: 0.3308
Epoch 2/10
17/17 [==============================] - 34s 2s/step - loss: 1.0983 - accuracy: 0.3487 - val_loss: 1.0955 - val_accuracy: 0.3308
Epoch 3/10
17/17 [==============================] - 34s 2s/step - loss: 1.0936 - accuracy: 0.3760 - val_loss: 1.0758 - val_accuracy: 0.4211
Epoch 4/10
17/17 [==============================] - 35s 2s/step - loss: 1.0755 - accuracy: 0.4063 - val_loss: 1.0431 - val_accuracy: 0.4361
Epoch 5/10
17/17 [==============================] - 35s 2s/step - loss: 1.0332 - accuracy: 0.4540 - val_loss: 0.9800 - val_accuracy: 0.4586
Epoch 6/10
17/17 [==============================] - 34s 2s/step - loss: 0.9570 - accuracy: 0.4745 - val_loss: 0.9273 - val_accuracy: 0.5338
Epoch 7/10
17/17 [==============================] - 35s 2s/step - loss: 0.9452 - accuracy: 0.5101 - val_loss: 0.8666 - val_accuracy: 0.5338
Epoch 8/10
17/17 [==

In [34]:
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity90.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity90.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
17/17 [==============================] - 40s 2s/step - loss: 1.2143 - accuracy: 0.3291 - val_loss: 1.1001 - val_accuracy: 0.3308
Epoch 2/10
17/17 [==============================] - 34s 2s/step - loss: 1.0959 - accuracy: 0.3395 - val_loss: 1.0986 - val_accuracy: 0.3308
Epoch 3/10
17/17 [==============================] - 35s 2s/step - loss: 1.0987 - accuracy: 0.3128 - val_loss: 1.0986 - val_accuracy: 0.3308
Epoch 4/10
17/17 [==============================] - 35s 2s/step - loss: 1.0984 - accuracy: 0.3608 - val_loss: 1.0988 - val_accuracy: 0.3308
Epoch 5/10
17/17 [==============================] - 35s 2s/step - loss: 1.0982 - accuracy: 0.3495 - val_loss: 1.0988 - val_accuracy: 0.3308
Epoch 6/10
17/17 [==============================] - 35s 2s/step - loss: 1.0987 - accuracy: 0.3319 - val_loss: 1.0987 - val_accuracy: 0.3308
Epoch 7/10
17/17 [==============================] - 34s 2s/step - loss: 1.0984 - accuracy: 0.3446 - val_loss: 1.0987 - val_accuracy: 0.3308
Epoch 8/10
17/17 [==

Epoch 1/10
17/17 [==============================] - 40s 2s/step - loss: 1.2462 - accuracy: 0.3455 - val_loss: 1.0970 - val_accuracy: 0.3308
Epoch 2/10
17/17 [==============================] - 34s 2s/step - loss: 1.0911 - accuracy: 0.3835 - val_loss: 1.1015 - val_accuracy: 0.3383
Epoch 3/10
17/17 [==============================] - 35s 2s/step - loss: 1.1050 - accuracy: 0.3287 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 4/10
17/17 [==============================] - 34s 2s/step - loss: 1.0992 - accuracy: 0.3292 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 5/10
17/17 [==============================] - 34s 2s/step - loss: 1.0984 - accuracy: 0.3452 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 6/10
17/17 [==============================] - 34s 2s/step - loss: 1.0988 - accuracy: 0.3412 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 7/10
17/17 [==============================] - 34s 2s/step - loss: 1.0985 - accuracy: 0.3448 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 8/10
17/17 [==

In [35]:
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_PolynomialDecay90.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_PolynomialDecay90.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
17/17 [==============================] - 42s 2s/step - loss: 1.2279 - accuracy: 0.3469 - val_loss: 1.1016 - val_accuracy: 0.3308
Epoch 2/10
17/17 [==============================] - 35s 2s/step - loss: 1.1000 - accuracy: 0.3353 - val_loss: 1.0985 - val_accuracy: 0.3308
Epoch 3/10
17/17 [==============================] - 35s 2s/step - loss: 1.0987 - accuracy: 0.3228 - val_loss: 1.0977 - val_accuracy: 0.3609
Epoch 4/10
17/17 [==============================] - 35s 2s/step - loss: 1.0930 - accuracy: 0.4115 - val_loss: 1.0990 - val_accuracy: 0.3308
Epoch 5/10
17/17 [==============================] - 36s 2s/step - loss: 1.1000 - accuracy: 0.3273 - val_loss: 1.0991 - val_accuracy: 0.3383
Epoch 6/10
17/17 [==============================] - 36s 2s/step - loss: 1.0988 - accuracy: 0.3444 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 7/10
17/17 [==============================] - 36s 2s/step - loss: 1.0986 - accuracy: 0.3289 - val_loss: 1.0988 - val_accuracy: 0.3308
Epoch 8/10
17/17 [==

Epoch 1/10
17/17 [==============================] - 41s 2s/step - loss: 1.2877 - accuracy: 0.3407 - val_loss: 1.1005 - val_accuracy: 0.3308
Epoch 2/10
17/17 [==============================] - 35s 2s/step - loss: 1.0995 - accuracy: 0.3174 - val_loss: 1.0986 - val_accuracy: 0.3308
Epoch 3/10
17/17 [==============================] - 35s 2s/step - loss: 1.0989 - accuracy: 0.2977 - val_loss: 1.0987 - val_accuracy: 0.3308
Epoch 4/10
17/17 [==============================] - 36s 2s/step - loss: 1.0986 - accuracy: 0.3357 - val_loss: 1.0987 - val_accuracy: 0.3308
Epoch 5/10
17/17 [==============================] - 35s 2s/step - loss: 1.0986 - accuracy: 0.3367 - val_loss: 1.0986 - val_accuracy: 0.3308
Epoch 6/10
17/17 [==============================] - 35s 2s/step - loss: 1.0987 - accuracy: 0.3238 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 7/10
17/17 [==============================] - 36s 2s/step - loss: 1.0987 - accuracy: 0.3313 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 8/10
17/17 [==

In [36]:
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_ConstantSparsity50.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_ConstantSparsity50.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

OSError: SavedModel file does not exist at: beans_models_optimized/EfficentNetB0_beans_model_ConstantSparsity50.h5/{saved_model.pbtxt|saved_model.pb}

In [37]:
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_PolynomialDecay50.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_PolynomialDecay50.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

OSError: SavedModel file does not exist at: beans_models_optimized/EfficentNetB0_beans_model_PolynomialDecay50.h5/{saved_model.pbtxt|saved_model.pb}

In [38]:
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_ConstantSparsity75.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_ConstantSparsity75.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

OSError: SavedModel file does not exist at: beans_models_optimized/EfficentNetB0_beans_model_ConstantSparsity75.h5/{saved_model.pbtxt|saved_model.pb}

In [39]:
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_PolynomialDecay75.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_PolynomialDecay75.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

OSError: SavedModel file does not exist at: beans_models_optimized/EfficentNetB0_beans_model_PolynomialDecay75.h5/{saved_model.pbtxt|saved_model.pb}

In [40]:
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_ConstantSparsity90.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_ConstantSparsity90.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

OSError: SavedModel file does not exist at: beans_models_optimized/EfficentNetB0_beans_model_ConstantSparsity90.h5/{saved_model.pbtxt|saved_model.pb}

In [41]:
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_PolynomialDecay90.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_PolynomialDecay90.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

OSError: SavedModel file does not exist at: beans_models_optimized/EfficentNetB0_beans_model_PolynomialDecay90.h5/{saved_model.pbtxt|saved_model.pb}

## Quantization of all models

In [42]:
# get all models paths
model_paths = []
import os
for file in os.listdir("beans_models/"):
    if file.endswith(".h5"):
        model_paths.append(str(os.path.join("beans_models/", file)))
for file in os.listdir("beans_models_optimized/"):
    if file.endswith(".h5"):
        model_paths.append(str(os.path.join("beans_models_optimized/", file)))
for file in os.listdir("flowers_models/"):
    if file.endswith(".h5"):
        model_paths.append(str(os.path.join("flowers_models/", file)))
for file in os.listdir("flowers_models_optimized/"):
    if file.endswith(".h5"):
        model_paths.append(str(os.path.join("flowers_models_optimized/", file)))

In [43]:
model_paths

['beans_models/EfficentNetB0_beans_model.h5',
 'beans_models/MobileNetV2_beans_model.h5',
 'beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity50_KMeansPlusPlus32.h5',
 'beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity90_KMeansPlusPlus32.h5',
 'beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity50_KMeansPlusPlus128.h5',
 'beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity90_KMeansPlusPlus128.h5',
 'beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity75.h5',
 'beans_models_optimized/MobileNetV2_beans_model_PolynomialDecay75_KMeansPlusPlus32.h5',
 'beans_models_optimized/MobileNetV2_beans_model_PolynomialDecay90.h5',
 'beans_models_optimized/MobileNetV2_beans_model_PolynomialDecay50_KMeansPlusPlus128.h5',
 'beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity75_KMeansPlusPlus32.h5',
 'beans_models_optimized/MobileNetV2_beans_model_PolynomialDecay50_KMeansPlusPlus32.h5',
 'beans_models_optimized/MobileNetV2_beans_mo

In [44]:
for model_path in model_paths:
    if "flowers" in model_path:
        model_quantization(model_path=model_path, ds=flowers_datasets[0])
    if "beans" in model_path:
        model_quantization(model_path=model_path, ds=beans_datasets[0])

INFO:tensorflow:Assets written to: /tmp/tmpn4ftvqq3/assets


INFO:tensorflow:Assets written to: /tmp/tmpn4ftvqq3/assets


Converted TFLite model (16044956 Bytes) saved to: beans_models_optimized/EfficentNetB0_beans_model.tflite
INFO:tensorflow:Assets written to: /tmp/tmpkzgwetic/assets


INFO:tensorflow:Assets written to: /tmp/tmpkzgwetic/assets


Dynamic range quantizatized TFLite model (4789504 Bytes) saved to: beans_models_optimized/EfficentNetB0_beans_model_dynamic_rage_quantization.tflite
INFO:tensorflow:Assets written to: /tmp/tmp3j3k4_nk/assets


...

Full integer quantizatized TFLite model (2975920 Bytes) saved to: flowers_models_optimized/MobileNetV2_flowers_model_ConstantSparsity90_KMeansPlusPlus128_full_integer_quantization.tflite
INFO:tensorflow:Assets written to: /tmp/tmpboiylcqw/assets


INFO:tensorflow:Assets written to: /tmp/tmpboiylcqw/assets


Full integer quantizatized with integer io TFLite model (2974680 Bytes) saved to: flowers_models_optimized/MobileNetV2_flowers_model_ConstantSparsity90_KMeansPlusPlus128_full_integer_quantization_integer_io.tflite
INFO:tensorflow:Assets written to: /tmp/tmpms5ff05y/assets


INFO:tensorflow:Assets written to: /tmp/tmpms5ff05y/assets


float16 quantizatized TFLite model (4723808 Bytes) saved to: flowers_models_optimized/MobileNetV2_flowers_model_ConstantSparsity90_KMeansPlusPlus128_float16_quantization.tflite
